In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

def carregar_dados(url, nomes_colunas):
    """Carrega os dados a partir de uma URL e retorna um DataFrame."""
    data = pd.read_csv(url, names=nomes_colunas, header=0)
    return data

def preprocessamento(X):
    """Realiza o pré-processamento dos dados."""
    # Selecionar apenas colunas numéricas
    numeric_columns = X.select_dtypes(include=[np.number]).columns
    X_numeric = X[numeric_columns]
    
    # Preencher valores ausentes
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X_numeric)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    return X_scaled

def encontrar_valor_k(X_scaled):
    """Encontra o valor ideal de K usando o método do cotovelo."""
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
        kmeans.fit(X_scaled)
        wcss.append(kmeans.inertia_)
    return wcss

def executar_kmeans(X_scaled, k):
    """Executa o algoritmo K-means com o valor de K especificado."""
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(X_scaled)
    y_kmeans = kmeans.predict(X_scaled)
    return y_kmeans

def executar_hierarquico(X_scaled, k, linkage):
    """Executa o algoritmo Hierárquico com o método de linkage especificado."""
    model = AgglomerativeClustering(n_clusters=k, linkage=linkage)
    y_hier = model.fit_predict(X_scaled)
    return y_hier

def calcular_medida_avaliacao_propria(labels, classes):
    """Calcula uma medida de avaliação própria para clusterização."""
    total = len(labels)
    prop_classes = []
    for c in classes:
        prop_classes.append(sum(labels == c) / total)
    return np.mean(prop_classes)

def comparar_kmeans_hierarquico(X_scaled, k, linkage):
    """Compara os resultados do K-means e Hierárquico."""
    # Executar K-means
    y_kmeans = executar_kmeans(X_scaled, k)

    # Executar Hierárquico
    y_hier = executar_hierarquico(X_scaled, k, linkage)

    # Comparação entre K-means e Hierárquico
    silhouette_kmeans = silhouette_score(X_scaled, y_kmeans)
    silhouette_hier = silhouette_score(X_scaled, y_hier)

    print(f"Comparação entre K-means e Hierárquico com linkage='{linkage}':")
    print("Silhouette Score K-means:", silhouette_kmeans)
    print("Silhouette Score Hierárquico:", silhouette_hier)
    print()

def executar_clusterizacao(url, nomes_colunas, k, linkage='ward'):
    """Função principal para executar a clusterização."""
    # Carregar os dados
    data = carregar_dados(url, nomes_colunas)
    X = data.drop(columns=['categoria_comportamento'])

    # Pré-processamento
    X_scaled = preprocessamento(X)

    # Encontrar valor de K usando o método do cotovelo
    wcss = encontrar_valor_k(X_scaled)

    # Escolher o valor de K
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 11), wcss, marker='o', linestyle='--', color='green')
    plt.title('Método do Cotovelo')
    plt.xlabel('Número de Clusters')
    plt.ylabel('WCSS')  # Within cluster sum of squares
    plt.xticks(np.arange(1, 11, 1))
    plt.grid(True)
    plt.show()

    # Executar K-means
    y_kmeans = executar_kmeans(X_scaled, k)

    # Executar Hierárquico
    y_hier = executar_hierarquico(X_scaled, k, linkage)

    # Comparar K-means e Hierárquico
    comparar_kmeans_hierarquico(X_scaled, k, linkage)

    # Avaliação própria
    classes = np.unique(y_kmeans)  # ou y_hier, ambos têm o mesmo número de clusters
    avaliacao_kmeans = calcular_medida_avaliacao_propria(y_kmeans, classes)
    avaliacao_hier = calcular_medida_avaliacao_propria(y_hier, classes)

    print("Avaliação própria para K-means:", avaliacao_kmeans)
    print("Avaliação própria para Hierárquico (linkage='ward'):", avaliacao_hier)

# URL dos dados e nomes das colunas
url = "https://raw.githubusercontent.com/tmfilho/akcdata/master/data/akc-data-latest.csv"
nomes_colunas = ['raça', 'descrição', 'temperamento', 'popularidade', 'altura_mínima', 'altura_máxima',
                 'peso_mínimo', 'peso_máximo', 'expectativa_vida_mínima', 'expectativa_vida_máxima', 'grupo',
                 'frequência_escovação_valor', 'frequência_escovação_categoria', 'queda_valor', 'queda_categoria',
                 'nível_energia_valor', 'nível_energia_categoria', 'facilidade_treinamento_valor', 'facilidade_treinamento_categoria',
                 'comportamento_valor', 'categoria_comportamento']

# Executar clusterização
executar_clusterizacao(url, nomes_colunas, k=3)


In [3]:
""" 
    Todas importações necessárias para o pré-processamento dos dados,
    o teste de Algoritmos e avaliação dos resultados
"""


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [4]:
"""
    Recebe como parâmetros a URL do conjunto de dados e os nomes das colunas.
    Lê o arquivo CSV utilizando o pandas e retorna um DataFrame com os dados.
"""

def carregar_dados(url, nomes_colunas):
    """Carrega os dados a partir de uma URL e retorna um DataFrame."""
    data = pd.read_csv(url, names=nomes_colunas, header=0)
    return data


"""
    Recebe o DataFrame de entrada (X).
    Seleciona apenas colunas numéricas.
    Imputa valores ausentes com estratégia median usando o SimpleImputer.
    Normaliza os dados com o StandardScaler.
    Retorna os dados pré-processados.
"""

def pre_processamento(X):
    """Realiza o pré-processamento dos dados."""
    # Selecionar apenas colunas numéricas
    numeric_columns = X.select_dtypes(include=[np.number]).columns
    X_numeric = X[numeric_columns]
    
    # Preencher valores ausentes
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X_numeric)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    return X_scaled


"""
    Nessa função faz os sguintes o seguinte:

    Recebe o DataFrame pré-processado (X_scaled).
    Calcula a WCSS (Within Cluster Sum of Squares) para cada valor de K entre 1 e 10.
    Retorna uma lista com os valores de WCSS.
"""

def encontrar_valor_k(X_scaled):
    """Encontra o valor ideal de K usando o método do cotovelo."""
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
        kmeans.fit(X_scaled)
        wcss.append(kmeans.inertia_)
    return wcss


"""
    Recebe o DataFrame pré-processado (X_scaled) e o valor de K.
    Executa o algoritmo K-means com o valor de K especificado.
    Retorna o vetor de rótulos de clusterização (y_kmeans).
"""

def executar_kmeans(X_scaled, k):
    """Executa o algoritmo K-means com o valor de K especificado."""
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(X_scaled)
    y_kmeans = kmeans.predict(X_scaled)
    return y_kmeans


"""
    Recebe o DataFrame pré-processado (X_scaled), o valor de K e o método de linkage.
    Executa o algoritmo Hierárquico com o valor de K e o método de linkage especificado.
    Retorna o vetor de rótulos de clusterização (y_hier).
"""

def executar_hierarquico(X_scaled, k, linkage):
    """Executa o algoritmo Hierárquico com o método de linkage especificado."""
    model = AgglomerativeClustering(n_clusters=k, linkage=linkage)
    y_hier = model.fit_predict(X_scaled)
    return y_hier


"""
    Recebe os vetores de rótulos de clusterização (labels) e as classes (classes).
    Calcula a proporção média de pontos em cada cluster.
    Retorna a proporção média de pontos em cada cluster.
"""

def calcular_medida_avaliacao_propria(labels, classes):
    """Calcula uma medida de avaliação própria para clusterização."""
    total = len(labels)
    prop_classes = []
    for c in classes:
        prop_classes.append(sum(labels == c) / total)
    return np.mean(prop_classes)


"""
    Recebe o DataFrame pré-processado (X_scaled), o valor de K e o método de linkage.
    Executa os algoritmos K-means e Hierárquico com o valor de K e o método de linkage especificado.
    Calcula o Silhouette Score para cada algoritmo.
    Exibe os resultados da comparação entre os algoritmos.
"""

def comparar_kmeans_hierarquico(X_scaled, k, linkage):
    """Compara os resultados do K-means e Hierárquico."""
    # Executar K-means
    y_kmeans = executar_kmeans(X_scaled, k)

    # Executar Hierárquico
    y_hier = executar_hierarquico(X_scaled, k, linkage)

    # Comparação entre K-means e Hierárquico
    silhouette_kmeans = silhouette_score(X_scaled, y_kmeans)
    silhouette_hier = silhouette_score(X_scaled, y_hier)

    print(f"Comparação entre K-means e Hierárquico com linkage='{linkage}':")
    print("Silhouette Score K-means:", silhouette_kmeans)
    print("Silhouette Score Hierárquico:", silhouette_hier)
    print()



"""
    Função principal para executar a clusterização.
    Carrega os dados, realiza o pré-processamento, encontra o valor ideal de K usando o método do cotovelo, executa os algoritmos K-means e Hierárquico, compara os resultados e calcula a avaliação própria.
"""

def executar_clusterizacao(url, nomes_colunas, k, linkage='ward'):
    """Função principal para executar a clusterização."""
    # Carregar os dados
    data = carregar_dados(url, nomes_colunas)
    X = data.drop(columns=['categoria_comportamento'])

    # Pré-processamento
    X_scaled = pre_processamento(X)

    # Encontrar valor de K usando o método do cotovelo
    wcss = encontrar_valor_k(X_scaled)

    # Escolher o valor de K
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 11), wcss, marker='o', linestyle='--', color='green')
    plt.title('Método do Cotovelo')
    plt.xlabel('Número de Clusters')
    plt.ylabel('WCSS')  # Within cluster sum of squares
    plt.xticks(np.arange(1, 11, 1))
    plt.grid(True)
    plt.show()

    # Executar K-means
    y_kmeans = executar_kmeans(X_scaled, k)

    # Executar Hierárquico
    y_hier = executar_hierarquico(X_scaled, k, linkage)

    # Comparar K-means e Hierárquico
    comparar_kmeans_hierarquico(X_scaled, k, linkage)

    # Avaliação própria
    classes = np.unique(y_kmeans)  # ou y_hier, ambos têm o mesmo número de clusters
    avaliacao_kmeans = calcular_medida_avaliacao_propria(y_kmeans, classes)
    avaliacao_hier = calcular_medida_avaliacao_propria(y_hier, classes)

    print("Avaliação própria para K-means:", avaliacao_kmeans)
    print("Avaliação própria para Hierárquico (linkage='ward'):", avaliacao_hier)


In [5]:
"""
    E por último esse trecho do código é onde é chamada a função de 
    clusterização, e motra os resultados.
"""

# URL dos dados e nomes das colunas
url = "https://raw.githubusercontent.com/tmfilho/akcdata/master/data/akc-data-latest.csv"
nomes_colunas = ['raça', 'descrição', 'temperamento', 'popularidade', 'altura_mínima', 'altura_máxima',
                 'peso_mínimo', 'peso_máximo', 'expectativa_vida_mínima', 'expectativa_vida_máxima', 'grupo',
                 'frequência_escovação_valor', 'frequência_escovação_categoria', 'queda_valor', 'queda_categoria',
                 'nível_energia_valor', 'nível_energia_categoria', 'facilidade_treinamento_valor', 'facilidade_treinamento_categoria',
                 'comportamento_valor', 'categoria_comportamento']

# Executar clusterização
executar_clusterizacao(url, nomes_colunas, k=3)

<class 'urllib.error.URLError'>: <urlopen error [Errno 23] Host is unreachable>